## Introduction to SageMaker TensorFlow - Image Classification

In [16]:
# THIS IS A SAMPLE OF USING AN AWS PRE-TRAINED MODEL
import sagemaker, boto3, json
from sagemaker.session import Session

sagemaker_session = Session()
aws_role = 'arn:aws:iam::478614889459:role/aebrain_sagemaker_role'
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

print(aws_role)


# Test s3 access
import boto3

s3_client = boto3.client('s3')
response = s3_client.list_buckets()

buckets = response['Buckets']
for bucket in buckets:
    print(bucket['Name'])

arn:aws:iam::478614889459:role/aebrain_sagemaker_role
sagemaker-ap-south-1-478614889459
web-resume-christian-echica
www.christianechica.cloud


In [17]:
# Assign a pre-trained model
model_id, model_version = "tensorflow-ic-imagenet-mobilenet-v2-100-224-classification-4", "*"

In [18]:
# To find another pre-trained model
import IPython
from ipywidgets import Dropdown
from sagemaker.jumpstart.notebook_utils import list_jumpstart_models
from sagemaker.jumpstart.filters import And

# Retrieves all TensorFlow Image Classification models made available by SageMaker Built-In Algorithms.
filter_value = And("task == ic", "framework == tensorflow")
ic_models = list_jumpstart_models(filter=filter_value)

# display the model-ids in a dropdown, for user to select a model.
dropdown = Dropdown(
    options=ic_models,
    value=model_id,
    description="SageMaker Built-In TensorFlow Image Classification Models:",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)
display(IPython.display.Markdown("## Select a SageMaker pre-trained model from the dropdown below"))
display(dropdown)

## Select a SageMaker pre-trained model from the dropdown below

Dropdown(description='SageMaker Built-In TensorFlow Image Classification Models:', index=106, layout=Layout(wi…

In [19]:
# Retrieve Artifacts and Deploy an Endpoint
from sagemaker import image_uris, model_uris, script_uris
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base

# model_version="*" fetches the latest version of the model.
infer_model_id, infer_model_version = dropdown.value, "*"

endpoint_name = name_from_base(f"jumpstart-example-{infer_model_id}")

# inference_instance_type = "ml.p2.xlarge" # GPU processing

inference_instance_type = "ml.m5.large" # CPU processing

# Retrieve the inference docker container uri.
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=infer_model_id,
    model_version=infer_model_version,
    instance_type=inference_instance_type,
)
# Retrieve the inference script uri.
deploy_source_uri = script_uris.retrieve(
    model_id=infer_model_id, model_version=infer_model_version, script_scope="inference"
)
# Retrieve the base model uri.
base_model_uri = model_uris.retrieve(
    model_id=infer_model_id, model_version=infer_model_version, model_scope="inference"
)
# Create the SageMaker model instance. Note that we need to pass Predictor class when we deploy model through Model class,
# for being able to run inference through the sagemaker API.
model = Model(
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    model_data=base_model_uri,
    entry_point="inference.py",
    role=aws_role,
    predictor_cls=Predictor,
    name=endpoint_name,
)
# deploy the Model.
base_model_predictor = model.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    endpoint_name=endpoint_name,
)

---!

In [ ]:
# Download example images locally from a public S3 bucket
s3_bucket = f"jumpstart-cache-prod-{aws_region}"
key_prefix = "inference-notebook-assets"


def download_from_s3(images):
    for filename, image_key in images.items():
        boto3.client("s3").download_file(s3_bucket, f"{key_prefix}/{image_key}", filename)


images = {"img1.jpg": "cat.jpg", "img2.jpg": "dog.jpg"}
download_from_s3(images)

In [20]:
# Test the endpoint to classify
from IPython.core.display import HTML


def predict_top_k_labels(probabilities, labels, k):
    topk_prediction_ids = sorted(
        range(len(probabilities)), key=lambda index: probabilities[index], reverse=True
    )[:k]
    topk_class_labels = ", ".join([labels[id] for id in topk_prediction_ids])
    return topk_class_labels


for image_filename in images.keys():
    with open(image_filename, "rb") as file:
        img = file.read()
    query_response = base_model_predictor.predict(
        img, {"ContentType": "application/x-image", "Accept": "application/json;verbose"}
    )
    model_predictions = json.loads(query_response)
    labels, probabilities = model_predictions["labels"], model_predictions["probabilities"]
    top5_class_labels = predict_top_k_labels(probabilities, labels, 5)
    display(
        HTML(
            f'<img src={image_filename} alt={image_filename} align="left" style="width: 250px;"/>'
            f"<figcaption>Top-5 predictions: {top5_class_labels} </figcaption>"
        )
    )

In [38]:
# Test the API
import boto3
import base64

# Function to predict top k labels
def predict_top_k_labels(probabilities, k):
    topk_prediction_ids = sorted(range(len(probabilities)), key=lambda index: probabilities[index], reverse=True)[:k]
    topk_class_labels = ", ".join([str(id) for id in topk_prediction_ids])
    return topk_class_labels

# Initialize the SageMaker runtime client
client = boto3.client('sagemaker-runtime', region_name='ap-south-1')

# Iterate over the images
for image_filename in images.keys():
    with open(image_filename, "rb") as file:
        img = file.read()

    # Convert image data to base64
    img_base64 = base64.b64encode(img).decode('utf-8')

    # Prepare the request payload
    payload = img

    # Send the API request
    response = client.invoke_endpoint(
        EndpointName='jumpstart-example-tensorflow-ic-imagene-2023-07-18-02-21-44-751',
        ContentType='application/x-image',
        Body=payload
    )

    # Check if the response was successful
    if response['ResponseMetadata']['HTTPStatusCode'] == 200:
        try:
            # Parse the response JSON
            response_body = response['Body'].read().decode('utf-8')
            print("Response JSON:", response_body)  # Print the response body for inspection

            # Parse the probabilities from the response
            probabilities = json.loads(response_body)['probabilities']

            # Process the predictions as per your requirement
            top5_class_labels = predict_top_k_labels(probabilities, 5)

            # Print the predictions
            print("Image:", image_filename)
            print("Top-5 predictions:", top5_class_labels)
            print()
        except KeyError as e:
            print("Error: Required key not found in the response JSON:", e)
        except json.JSONDecodeError as e:
            print("Error: Failed to decode the response JSON:", e)
    else:
        print("Error:", response['ResponseMetadata']['HTTPStatusCode'], response['Body'].read())


Response JSON: {"probabilities": [0.000220340167, 0.00027350086, 0.00134269183, 0.000143566358, 0.0003840914, 0.000570199802, 0.000105629573, 0.000127337174, 0.000112846763, 0.000440019794, 6.81042293e-05, 9.39032907e-05, 6.89746739e-05, 0.000199663176, 0.000355220225, 0.000187777114, 0.000463004078, 0.000179361276, 0.000326805224, 5.6562465e-05, 0.000132113055, 0.000155819012, 0.000209577527, 7.37654482e-05, 0.000112373484, 0.000272623263, 0.000187207086, 9.39568563e-05, 0.000331980467, 0.000167981561, 0.000150269552, 0.00015549791, 0.000484081305, 0.000326994137, 0.000122802099, 6.11904616e-05, 0.000132737288, 0.000100230252, 0.000192133, 0.000237547807, 0.000664034043, 0.00025130529, 0.000268641335, 0.000332894793, 0.000172213433, 5.16639775e-05, 0.000199302856, 0.000306557165, 0.000135241251, 4.03381455e-05, 3.37958045e-05, 6.30448267e-05, 0.00067490869, 0.000242128401, 0.000179086273, 0.000245316041, 0.000214729807, 0.000170240208, 0.00016595787, 0.000189175247, 0.00013990869, 5.5

Response JSON: {"probabilities": [0.000275432045, 0.000291630888, 0.000371928851, 0.000209135964, 0.000678849872, 0.000728412124, 0.000326798501, 0.000192822437, 0.000740698364, 0.000191767147, 0.000128168263, 0.000100259545, 0.000243637332, 0.000290352938, 0.00045907704, 0.000171980442, 9.89786204e-05, 0.000154328285, 0.000300734333, 0.00017996387, 0.000403960439, 0.00017230575, 0.000233574974, 0.000134205853, 8.10705606e-05, 0.000429464882, 0.000116520365, 6.22233056e-05, 0.00014919629, 0.000701224606, 0.000466315134, 0.000369740446, 0.000158950061, 0.000260585977, 5.45719085e-05, 0.000162023643, 0.000476325484, 9.18463265e-05, 0.000359439466, 0.000110368092, 0.000210416445, 0.000513445644, 0.000113310307, 9.78858661e-05, 8.69079449e-05, 0.000384423038, 7.33696434e-05, 0.000431077904, 9.3453782e-05, 6.86625644e-05, 0.000298692379, 0.000184830467, 0.000151316533, 0.0002150902, 0.000150503023, 8.81010201e-05, 5.9726739e-05, 0.000135285663, 0.000330867857, 0.000128036685, 0.000195161323

In [40]:
# Test API again
import boto3
import base64

# Function to predict top k labels
def predict_top_k_labels(probabilities, k):
    topk_prediction_ids = sorted(range(len(probabilities)), key=lambda index: probabilities[index], reverse=True)[:k]
    topk_class_labels = [str(id) for id in topk_prediction_ids]
    return topk_class_labels

# Initialize the SageMaker runtime client
client = boto3.client('sagemaker-runtime', region_name='ap-south-1')

# Iterate over the images
for image_filename in images.keys():
    with open(image_filename, "rb") as file:
        img = file.read()

    # Convert image data to base64
    img_base64 = base64.b64encode(img).decode('utf-8')

    # Prepare the request payload
    payload = img

    # Send the API request
    response = client.invoke_endpoint(
        EndpointName='jumpstart-example-tensorflow-ic-imagene-2023-07-18-02-21-44-751',
        ContentType='application/x-image',
        Body=payload
    )

    # Check if the response was successful
    if response['ResponseMetadata']['HTTPStatusCode'] == 200:
        try:
            # Parse the response JSON
            response_body = response['Body'].read().decode('utf-8')
            print("Response JSON:", response_body)  # Print the response body for inspection

            # Process the predictions as per your requirement
            response_data = json.loads(response_body)
            if 'probabilities' in response_data:
                probabilities = response_data['probabilities']
                top5_class_labels = predict_top_k_labels(probabilities, 5)
                print("Image:", image_filename)
                print("Top-5 predictions:", ", ".join(top5_class_labels))
                print()
            else:
                print("Error: 'probabilities' key not found in the response JSON")
        except KeyError as e:
            print("Error: Required key not found in the response JSON:", e)
        except json.JSONDecodeError as e:
            print("Error: Failed to decode the response JSON:", e)
    else:
        print("Error:", response['ResponseMetadata']['HTTPStatusCode'], response['Body'].read())


Response JSON: {"probabilities": [0.000220340167, 0.00027350086, 0.00134269183, 0.000143566358, 0.0003840914, 0.000570199802, 0.000105629573, 0.000127337174, 0.000112846763, 0.000440019794, 6.81042293e-05, 9.39032907e-05, 6.89746739e-05, 0.000199663176, 0.000355220225, 0.000187777114, 0.000463004078, 0.000179361276, 0.000326805224, 5.6562465e-05, 0.000132113055, 0.000155819012, 0.000209577527, 7.37654482e-05, 0.000112373484, 0.000272623263, 0.000187207086, 9.39568563e-05, 0.000331980467, 0.000167981561, 0.000150269552, 0.00015549791, 0.000484081305, 0.000326994137, 0.000122802099, 6.11904616e-05, 0.000132737288, 0.000100230252, 0.000192133, 0.000237547807, 0.000664034043, 0.00025130529, 0.000268641335, 0.000332894793, 0.000172213433, 5.16639775e-05, 0.000199302856, 0.000306557165, 0.000135241251, 4.03381455e-05, 3.37958045e-05, 6.30448267e-05, 0.00067490869, 0.000242128401, 0.000179086273, 0.000245316041, 0.000214729807, 0.000170240208, 0.00016595787, 0.000189175247, 0.00013990869, 5.5

Response JSON: {"probabilities": [0.000275432045, 0.000291630888, 0.000371928851, 0.000209135964, 0.000678849872, 0.000728412124, 0.000326798501, 0.000192822437, 0.000740698364, 0.000191767147, 0.000128168263, 0.000100259545, 0.000243637332, 0.000290352938, 0.00045907704, 0.000171980442, 9.89786204e-05, 0.000154328285, 0.000300734333, 0.00017996387, 0.000403960439, 0.00017230575, 0.000233574974, 0.000134205853, 8.10705606e-05, 0.000429464882, 0.000116520365, 6.22233056e-05, 0.00014919629, 0.000701224606, 0.000466315134, 0.000369740446, 0.000158950061, 0.000260585977, 5.45719085e-05, 0.000162023643, 0.000476325484, 9.18463265e-05, 0.000359439466, 0.000110368092, 0.000210416445, 0.000513445644, 0.000113310307, 9.78858661e-05, 8.69079449e-05, 0.000384423038, 7.33696434e-05, 0.000431077904, 9.3453782e-05, 6.86625644e-05, 0.000298692379, 0.000184830467, 0.000151316533, 0.0002150902, 0.000150503023, 8.81010201e-05, 5.9726739e-05, 0.000135285663, 0.000330867857, 0.000128036685, 0.000195161323